# Reinforecement Learning for Traffic Light Control

Required packages or environments:

* Python 3.6
* SUMO 0.32 with TraCI module. Click the [link](https://sumo.dlr.de/docs/SUMO_User_Documentation.html) for user documentation 
* Keras 2.2.0 and Tensorflow 1.9.0

## Setting Up the environment through SUMO and TRACI

In [1]:
from __future__ import absolute_import
from __future__ import print_function

import os
import sys
import optparse
import argparse
import random

### Import python modules from the SUMO_HOME/tools directory

In [2]:
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

### Simulation environment setting up

In [3]:
from sumolib import checkBinary  
import traci  # noqa
import traci.constants as tc

### Generating route file (data/road.rou.xml) based on the demand per second from differnt directions

In [4]:
def generate_routefile():
    random.seed(42)  # make tests reproducible
    N = 3600  # number of time steps
    # demand per second from different directions
    pEB = 1. / 10
    pWB = 1. / 11
    pSB = 1. / 15
    pNB = 1. / 25
    with open("data/road.rou.xml", "w") as routes:
        print("""<routes>
        <vType id="typeWE" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2.5" maxSpeed="16.67" \
guiShape="passenger"/>
        <vType id="typeNS" accel="0.8" decel="4.5" sigma="0.5" length="7" minGap="3" maxSpeed="25" guiShape="bus"/>

        <route id="EastBound" edges="W-0W 0W-0 0-0E 0E-E" />
        <route id="WestBound" edges="E-0E 0E-0 0-0W 0W-W" />
        <route id="SouthBound" edges="N-0N 0N-0 0-0S 0S-S" />
        <route id="NorthBound" edges="S-0S 0S-0 0-0N 0N-N" />""", file=routes)
        vehNr = 0
        for i in range(N):
            if random.uniform(0, 1) < pEB:
                print('    <vehicle id="EastBound_%i" type="typeWE" route="EastBound" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pWB:
                print('    <vehicle id="WestBound_%i" type="typeWE" route="WestBound" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pSB:
                print('    <vehicle id="SouthBound_%i" type="typeNS" route="SouthBound" depart="%i" color="1,0,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pNB:
                print('    <vehicle id="NorthBound_%i" type="typeNS" route="NorthBound" depart="%i" color="1,0,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
        print("</routes>", file=routes)

In [5]:
# The program looks like this
#    <tlLogic id="tl0" type="static" programID="0" offset="0">
# the locations of the tls are      NESW
#        <phase duration="41" state="GGrrGGrr"/>
#        <phase duration="4" state="yyrryyrr"/>
#        <phase duration="41" state="rrGGrrGG"/>
#        <phase duration="4" state="rryyrryy"/>
#    </tlLogic>

In [6]:
def run():
    """execute the TraCI control loop"""
#    step = 0
    # we start with phase 2 where EW has green
   
    traci.junction.subscribeContext("tl0", tc.CMD_GET_VEHICLE_VARIABLE, 300, [tc.VAR_SPEED, tc.VAR_WAITING_TIME]) 
    print(traci.junction.getContextSubscriptionResults("tl0"))
#   traci.trafficlight.setPhase("tl0", 2)
#    while traci.lane.getLastStepVehicleNumber("0W-0_0") <200:
#   while traci.simulation.getMinExpectedNumber() > 0:        
    for step in range(300):
        print("step ", step)
        print( traci.simulation.getDeltaT(), "from last step")
        print("# of vechiles in and to come: ", traci.simulation.getMinExpectedNumber())
        print("Traffic Light is in phase ", traci.trafficlight.getPhase("tl0"),
              "which is as ", traci.trafficlight.getRedYellowGreenState("tl0"),
              "with duration of ", traci.trafficlight.getPhaseDuration("tl0"),
              "The next phase will start at timestep", traci.trafficlight.getNextSwitch("tl0"),"secends")
        traci.simulationStep()
        print(traci.edge.getLastStepVehicleNumber("0W-0"),"veichles are in eastbound approaching the TL",
              "and the # of veichles in waiting are ", traci.edge.getLastStepHaltingNumber("0W-0"),
              "Eastbound total waiting time is ", traci.edge.getWaitingTime("0W-0"))
        print(traci.junction.getContextSubscriptionResults("tl0"))
#    if traci.trafficlight.getPhase("tl0") == 2:
            # we are not already switching
#           if traci.inductionloop.getLastStepVehicleNumber("tl0") > 0:
                # there is a vehicle from the north, switch
#       traci.trafficlight.setPhase("tl0", 3)
#           else:
                # otherwise try to keep green for EW
#               traci.trafficlight.setPhase("tl0", 2)
#        eb = traci.lane.getLastStepVehicleNumber("0W-0_0")
#        step += 1
#        print("Step "+ str(step))
#        print('Eastbound vechicles : ', eb)
    traci.close()
    sys.stdout.flush()


In [7]:
import argparse
def get_options():
    optParser = argparse.ArgumentParser()
    optParser.add_argument("--nogui", action="store_true",
                         default=False, help="run the commandline version of sumo")
#    options, args = optParser.parse_args()
    options, args = optParser.parse_known_args()
#    optParser.print_help()
    return options

this is the main entry point of the simulation

In [8]:
if __name__ == "__main__":
    options = get_options()
#    print(options)
    # this script has been called from the command line. It will start sumo as a
    # server, then connect and run
    if options.nogui:
        sumoBinary = checkBinary('sumo')
    else:
        sumoBinary = checkBinary('sumo-gui')

    # first, generate the route file for this simulation
    generate_routefile()

    # this is the normal way of using traci. sumo is started as a
    # subprocess and then the python script connects and runs
    traci.start([sumoBinary, "-c", "data/road.sumocfg",
                             "--tripinfo-output", "tripinfo.xml"])
    run()

None
step  0
1.0 from last step
# of vechiles in and to come:  2
Traffic Light is in phase  0 which is as  GGrrGGrr with duration of  41.0 The next phase will start at timestep 41.0 secends
0 veichles are in eastbound approaching the TL and the # of veichles in waiting are  0 Eastbound total waiting time is  0.0
None
step  1
1.0 from last step
# of vechiles in and to come:  2
Traffic Light is in phase  0 which is as  GGrrGGrr with duration of  41.0 The next phase will start at timestep 41.0 secends
0 veichles are in eastbound approaching the TL and the # of veichles in waiting are  0 Eastbound total waiting time is  0.0
None
step  2
1.0 from last step
# of vechiles in and to come:  2
Traffic Light is in phase  0 which is as  GGrrGGrr with duration of  41.0 The next phase will start at timestep 41.0 secends
0 veichles are in eastbound approaching the TL and the # of veichles in waiting are  0 Eastbound total waiting time is  0.0
None
step  3
1.0 from last step
# of vechiles in and to co

4 veichles are in eastbound approaching the TL and the # of veichles in waiting are  0 Eastbound total waiting time is  0.0
{'EastBound_10': {64: 3.66525348769501, 122: 0.0}, 'EastBound_2': {64: 9.641704895948646, 122: 0.0}, 'EastBound_5': {64: 13.393299750611192, 122: 0.0}, 'EastBound_7': {64: 7.434420823305844, 122: 0.0}, 'NorthBound_3': {64: 17.879593385716372, 122: 0.0}, 'NorthBound_6': {64: 10.681618568766867, 122: 0.0}, 'SouthBound_12': {64: 2.025846062600613, 122: 0.0}, 'SouthBound_9': {64: 6.886493394710123, 122: 0.0}, 'WestBound_0': {64: 0.008100103459723403, 122: 1.0}, 'WestBound_1': {64: 15.422364918712514, 122: 0.0}, 'WestBound_4': {64: 13.622203475981953, 122: 0.0}, 'WestBound_8': {64: 7.362782284338027, 122: 0.0}}
step  35
1.0 from last step
# of vechiles in and to come:  54
Traffic Light is in phase  0 which is as  GGrrGGrr with duration of  41.0 The next phase will start at timestep 41.0 secends
4 veichles are in eastbound approaching the TL and the # of veichles in wai

# of vechiles in and to come:  52
Traffic Light is in phase  2 which is as  rrGGrrGG with duration of  41.0 The next phase will start at timestep 86.0 secends
4 veichles are in eastbound approaching the TL and the # of veichles in waiting are  0 Eastbound total waiting time is  0.0
{'EastBound_10': {64: 16.617980497777463, 122: 0.0}, 'EastBound_11': {64: 16.51266416812316, 122: 0.0}, 'EastBound_13': {64: 16.451300077959896, 122: 0.0}, 'EastBound_14': {64: 16.196895390829045, 122: 0.0}, 'EastBound_16': {64: 14.537595083541184, 122: 0.0}, 'EastBound_18': {64: 11.673505965620285, 122: 0.0}, 'EastBound_19': {64: 7.985378714650868, 122: 0.0}, 'EastBound_20': {64: 3.8636484946669776, 122: 0.0}, 'NorthBound_6': {64: 0.0, 122: 27.0}, 'SouthBound_12': {64: 0.0, 122: 11.0}, 'SouthBound_22': {64: 2.301246553473175, 122: 0.0}, 'SouthBound_9': {64: 0.0, 122: 19.0}, 'WestBound_0': {64: 16.54884508566931, 122: 0.0}, 'WestBound_1': {64: 15.440670756541179, 122: 0.0}, 'WestBound_15': {64: 15.3672439404

1.0 from last step
# of vechiles in and to come:  39
Traffic Light is in phase  0 which is as  GGrrGGrr with duration of  41.0 The next phase will start at timestep 131.0 secends
5 veichles are in eastbound approaching the TL and the # of veichles in waiting are  3 Eastbound total waiting time is  57.0
{'EastBound_18': {64: 0.0, 122: 25.0}, 'EastBound_19': {64: 0.0, 122: 19.0}, 'EastBound_20': {64: 0.0, 122: 13.0}, 'EastBound_24': {64: 2.9008977052807485, 122: 0.0}, 'EastBound_26': {64: 12.994261322543032, 122: 0.0}, 'NorthBound_28': {64: 7.310488183796405, 122: 0.0}, 'NorthBound_29': {64: 3.4710092981345957, 122: 0.0}, 'NorthBound_6': {64: 14.569872396625586, 122: 0.0}, 'SouthBound_12': {64: 15.199657023418702, 122: 0.0}, 'SouthBound_22': {64: 16.15432713941745, 122: 0.0}, 'SouthBound_25': {64: 14.534067552807032, 122: 0.0}, 'SouthBound_27': {64: 10.252599690668285, 122: 0.0}, 'SouthBound_9': {64: 14.046423756507743, 122: 0.0}, 'WestBound_21': {64: 0.0, 122: 16.0}, 'WestBound_23': {64

3 veichles are in eastbound approaching the TL and the # of veichles in waiting are  0 Eastbound total waiting time is  0.0
{'EastBound_18': {64: 12.297815026249738, 122: 0.0}, 'EastBound_19': {64: 11.919317791517827, 122: 0.0}, 'EastBound_20': {64: 11.160244354989926, 122: 0.0}, 'EastBound_24': {64: 10.88014394566393, 122: 0.0}, 'EastBound_26': {64: 9.925224095328666, 122: 0.0}, 'EastBound_30': {64: 10.843249105217101, 122: 0.0}, 'EastBound_35': {64: 15.057990237604836, 122: 0.0}, 'EastBound_36': {64: 13.24815676026047, 122: 0.0}, 'EastBound_39': {64: 10.522766663599764, 122: 0.0}, 'NorthBound_28': {64: 0.0, 122: 16.0}, 'NorthBound_29': {64: 0.0, 122: 10.0}, 'NorthBound_40': {64: 11.401603313907984, 122: 0.0}, 'SouthBound_38': {64: 13.82258794698865, 122: 0.0}, 'SouthBound_43': {64: 2.780963446665555, 122: 0.0}, 'WestBound_21': {64: 11.206048913579433, 122: 0.0}, 'WestBound_23': {64: 11.64347264710814, 122: 0.0}, 'WestBound_31': {64: 14.787275378538927, 122: 0.0}, 'WestBound_32': {64:

Traffic Light is in phase  0 which is as  GGrrGGrr with duration of  41.0 The next phase will start at timestep 221.0 secends
6 veichles are in eastbound approaching the TL and the # of veichles in waiting are  2 Eastbound total waiting time is  13.0
{'EastBound_44': {64: 0.0, 122: 9.0}, 'EastBound_45': {64: 0.0, 122: 4.0}, 'EastBound_47': {64: 14.757120983952767, 122: 0.0}, 'EastBound_49': {64: 12.567001628223812, 122: 0.0}, 'EastBound_50': {64: 7.820059322658924, 122: 0.0}, 'EastBound_51': {64: 4.954631929006427, 122: 0.0}, 'NorthBound_28': {64: 8.165934736281631, 122: 0.0}, 'NorthBound_29': {64: 7.848051021806896, 122: 0.0}, 'NorthBound_40': {64: 6.459131347279462, 122: 0.0}, 'SouthBound_38': {64: 8.124087467044593, 122: 0.0}, 'SouthBound_43': {64: 17.045798760368175, 122: 0.0}, 'SouthBound_48': {64: 14.453958208172073, 122: 0.0}, 'WestBound_41': {64: 15.741431153692885, 122: 0.0}, 'WestBound_42': {64: 0.0, 122: 10.0}, 'WestBound_46': {64: 5.305077935871639, 122: 0.0}}
step  193
1.0

7 veichles are in eastbound approaching the TL and the # of veichles in waiting are  1 Eastbound total waiting time is  6.0
{'EastBound_44': {64: 1.7538889683783057, 122: 0.0}, 'EastBound_45': {64: 1.7108228967525068, 122: 0.0}, 'EastBound_47': {64: 1.0628075450698011, 122: 0.0}, 'EastBound_49': {64: 1.180375895795919, 122: 0.0}, 'EastBound_50': {64: 0.33869058961642745, 122: 0.0}, 'EastBound_51': {64: 0.2921242114571408, 122: 0.0}, 'EastBound_52': {64: 0.0, 122: 6.0}, 'EastBound_56': {64: 8.637392295058817, 122: 0.0}, 'EastBound_57': {64: 5.330204517859965, 122: 0.0}, 'NorthBound_55': {64: 9.484583139978351, 122: 0.0}, 'SouthBound_54': {64: 12.783563967514791, 122: 0.0}, 'SouthBound_59': {64: 4.217624246608467, 122: 0.0}, 'WestBound_42': {64: 1.3270670218393208, 122: 0.0}, 'WestBound_46': {64: 1.8294286923483014, 122: 0.0}, 'WestBound_53': {64: 13.815331607498232, 122: 0.0}, 'WestBound_58': {64: 5.3021016931161284, 122: 0.0}}
step  228
1.0 from last step
# of vechiles in and to come: 

3 veichles are in eastbound approaching the TL and the # of veichles in waiting are  0 Eastbound total waiting time is  0.0
{'EastBound_57': {64: 16.307937277369202, 122: 0.0}, 'EastBound_60': {64: 16.360032606478782, 122: 0.0}, 'EastBound_62': {64: 16.44380206426606, 122: 0.0}, 'EastBound_64': {64: 12.45344616984949, 122: 0.0}, 'EastBound_66': {64: 9.504297249298546, 122: 0.0}, 'NorthBound_55': {64: 0.0, 122: 23.0}, 'NorthBound_65': {64: 11.179150922875854, 122: 0.0}, 'SouthBound_54': {64: 0.0, 122: 28.0}, 'SouthBound_59': {64: 0.0, 122: 13.0}, 'SouthBound_63': {64: 16.116996080573156, 122: 0.0}, 'SouthBound_68': {64: 6.19019100088626, 122: 0.0}, 'SouthBound_70': {64: 3.5648541672155267, 122: 0.0}, 'WestBound_58': {64: 16.44007945906371, 122: 0.0}, 'WestBound_61': {64: 16.58759316241369, 122: 0.0}, 'WestBound_67': {64: 8.419085399992763, 122: 0.0}, 'WestBound_69': {64: 5.223618113156409, 122: 0.0}, 'WestBound_71': {64: 2.2316432068124414, 122: 0.0}}
step  268
1.0 from last step
# of v